In [0]:
%python
from pyspark.sql.functions import date_format, col, to_date, round

# Wczytanie danych i połączenie
silver_df = (spark.read.format("delta").table("plstocks.bronze_prices_raw")
             .drop("<PER>", "<OPENINT>")
             .join(spark.read.format("delta").table("plstocks.companynamelookup"),
                   col("<TICKER>") == col("ticker"), "left")
             .drop("ticker"))

# Przekształcenie kolumny <DATE> i zmiana nazw kolumn
silver_df = (silver_df.withColumn("DATE", date_format(to_date(col("<DATE>").cast("string"), "yyyyMMdd"), "dd-MM-yyyy"))
             .withColumnRenamed("<TICKER>", "TICKER")
             .withColumnRenamed("stock_name", "STOCK_NAME")
             .withColumnRenamed("<TIME>", "TIME")
             .withColumnRenamed("<OPEN>", "OPEN")
             .withColumnRenamed("<HIGH>", "HIGH")
             .withColumnRenamed("<LOW>", "LOW")
             .withColumnRenamed("<CLOSE>", "CLOSE")
             .withColumnRenamed("<VOL>", "VOL"))

# Formatowanie wartości kolumn
for col_name in ["OPEN", "HIGH", "LOW", "CLOSE"]:
    silver_df = silver_df.withColumn(col_name, round(col(col_name), 2).cast("double"))

# Wybór kolumn w odpowiedniej kolejności
silver_df = silver_df.select("TICKER", "STOCK_NAME", "DATE", "TIME", "OPEN", "HIGH", "LOW", "CLOSE", "VOL")

display(silver_df)